# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 7 2022 11:42AM,241306,102,PBR-062422-01,"Senseonics, Incorporated",Released
1,Jul 7 2022 11:38AM,241305,19,8243578,"Emersa Waterbox, LLC",Released
2,Jul 7 2022 11:21AM,241299,10,8243547,Yusen – 3D Matrix,Released
3,Jul 7 2022 11:16AM,241298,10,6684252,Livs Products,Released
4,Jul 7 2022 11:16AM,241297,102,PBR-062422-02,"Senseonics, Incorporated",Released
5,Jul 7 2022 11:03AM,241295,10,MSP211935,"Methapharm, Inc.",Released
6,Jul 7 2022 11:03AM,241295,10,MSP211936,"Methapharm, Inc.",Released
7,Jul 7 2022 11:03AM,241295,10,MSP211937,"Methapharm, Inc.",Released
8,Jul 7 2022 11:03AM,241295,10,MSP211945,"Methapharm, Inc.",Released
9,Jul 7 2022 11:03AM,241295,10,MSP211948,"Methapharm, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,241297,Released,1
49,241298,Released,1
50,241299,Released,1
51,241305,Released,1
52,241306,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
241297,NaN,1.0
241298,NaN,1.0
241299,NaN,1.0
241305,NaN,1.0
241306,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
241208,0.0,1.0
241213,1.0,0.0
241224,3.0,54.0
241228,0.0,1.0
241240,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
241208,0,1
241213,1,0
241224,3,54
241228,0,1
241240,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,241208,0,1
1,241213,1,0
2,241224,3,54
3,241228,0,1
4,241240,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,241208,,1
1,241213,1,
2,241224,3,54
3,241228,,1
4,241240,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 7 2022 11:42AM,241306,102,"Senseonics, Incorporated"
1,Jul 7 2022 11:38AM,241305,19,"Emersa Waterbox, LLC"
2,Jul 7 2022 11:21AM,241299,10,Yusen – 3D Matrix
3,Jul 7 2022 11:16AM,241298,10,Livs Products
4,Jul 7 2022 11:16AM,241297,102,"Senseonics, Incorporated"
5,Jul 7 2022 11:03AM,241295,10,"Methapharm, Inc."
10,Jul 7 2022 10:40AM,241291,19,"Emersa Waterbox, LLC"
11,Jul 7 2022 10:28AM,241290,19,"Emersa Waterbox, LLC"
12,Jul 7 2022 10:25AM,241289,19,"Emersa Waterbox, LLC"
13,Jul 7 2022 10:24AM,241288,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jul 7 2022 11:42AM,241306,102,"Senseonics, Incorporated",,1
1,Jul 7 2022 11:38AM,241305,19,"Emersa Waterbox, LLC",,1
2,Jul 7 2022 11:21AM,241299,10,Yusen – 3D Matrix,,1
3,Jul 7 2022 11:16AM,241298,10,Livs Products,,1
4,Jul 7 2022 11:16AM,241297,102,"Senseonics, Incorporated",,1
5,Jul 7 2022 11:03AM,241295,10,"Methapharm, Inc.",,5
6,Jul 7 2022 10:40AM,241291,19,"Emersa Waterbox, LLC",,1
7,Jul 7 2022 10:28AM,241290,19,"Emersa Waterbox, LLC",,1
8,Jul 7 2022 10:25AM,241289,19,"Emersa Waterbox, LLC",,1
9,Jul 7 2022 10:24AM,241288,10,Eywa Pharma Inc.,,20


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 11:42AM,241306,102,"Senseonics, Incorporated",1,
1,Jul 7 2022 11:38AM,241305,19,"Emersa Waterbox, LLC",1,
2,Jul 7 2022 11:21AM,241299,10,Yusen – 3D Matrix,1,
3,Jul 7 2022 11:16AM,241298,10,Livs Products,1,
4,Jul 7 2022 11:16AM,241297,102,"Senseonics, Incorporated",1,
5,Jul 7 2022 11:03AM,241295,10,"Methapharm, Inc.",5,
6,Jul 7 2022 10:40AM,241291,19,"Emersa Waterbox, LLC",1,
7,Jul 7 2022 10:28AM,241290,19,"Emersa Waterbox, LLC",1,
8,Jul 7 2022 10:25AM,241289,19,"Emersa Waterbox, LLC",1,
9,Jul 7 2022 10:24AM,241288,10,Eywa Pharma Inc.,20,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 11:42AM,241306,102,"Senseonics, Incorporated",1,
1,Jul 7 2022 11:38AM,241305,19,"Emersa Waterbox, LLC",1,
2,Jul 7 2022 11:21AM,241299,10,Yusen – 3D Matrix,1,
3,Jul 7 2022 11:16AM,241298,10,Livs Products,1,
4,Jul 7 2022 11:16AM,241297,102,"Senseonics, Incorporated",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 11:42AM,241306,102,"Senseonics, Incorporated",1.0,NaN
1,Jul 7 2022 11:38AM,241305,19,"Emersa Waterbox, LLC",1.0,NaN
2,Jul 7 2022 11:21AM,241299,10,Yusen – 3D Matrix,1.0,NaN
3,Jul 7 2022 11:16AM,241298,10,Livs Products,1.0,NaN
4,Jul 7 2022 11:16AM,241297,102,"Senseonics, Incorporated",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 7 2022 11:42AM,241306,102,"Senseonics, Incorporated",1.0,0.0
1,Jul 7 2022 11:38AM,241305,19,"Emersa Waterbox, LLC",1.0,0.0
2,Jul 7 2022 11:21AM,241299,10,Yusen – 3D Matrix,1.0,0.0
3,Jul 7 2022 11:16AM,241298,10,Livs Products,1.0,0.0
4,Jul 7 2022 11:16AM,241297,102,"Senseonics, Incorporated",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5790462,302.0,0.0
102,482603,2.0,0.0
12,241208,1.0,0.0
15,965076,64.0,27.0
16,723709,1.0,2.0
19,1447657,64.0,3.0
20,482564,23.0,2.0
21,1447485,6.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5790462,302.0,0.0
1,102,482603,2.0,0.0
2,12,241208,1.0,0.0
3,15,965076,64.0,27.0
4,16,723709,1.0,2.0
5,19,1447657,64.0,3.0
6,20,482564,23.0,2.0
7,21,1447485,6.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,302.0,0.0
1,102,2.0,0.0
2,12,1.0,0.0
3,15,64.0,27.0
4,16,1.0,2.0
5,19,64.0,3.0
6,20,23.0,2.0
7,21,6.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,302.0
1,102,Released,2.0
2,12,Released,1.0
3,15,Released,64.0
4,16,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,102,12,15,16,19,20,21
Status,,,,,,,,
Executing,0.0,0.0,0.0,27.0,2.0,3.0,2.0,0.0
Released,302.0,2.0,1.0,64.0,1.0,64.0,23.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,102,12,15,16,19,20,21
0,Executing,0.0,0.0,0.0,27.0,2.0,3.0,2.0,0.0
1,Released,302.0,2.0,1.0,64.0,1.0,64.0,23.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,102,12,15,16,19,20,21
0,Executing,0.0,0.0,0.0,27.0,2.0,3.0,2.0,0.0
1,Released,302.0,2.0,1.0,64.0,1.0,64.0,23.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()